In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('df_interpolated (3).csv')  # dataset path
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# Rename columns for Prophet
df = df.rename(columns={'date': 'ds', 'gold_lkr': 'y'})

# Prepare the regressor variables
regressors = ['gold_price_usd', 'silver_price', 's&p_500_index', 'nyse_com_index', 'usd_selling_exrate', 'gold_futures', 'effr']

# Initialize and fit the Prophet model to capture the trend and seasonality
prophet_model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True)

# Add each independent variable as a regressor in Prophet
for regressor in regressors:
    prophet_model.add_regressor(regressor)

# Fit the Prophet model on the training data
prophet_model.fit(df[['ds', 'y'] + regressors])

# Make predictions using the Prophet model
df_forecast = prophet_model.predict(df[['ds'] + regressors])

# Calculate the residuals (actual - predicted by Prophet)
df['residuals'] = df['y'] - df_forecast['yhat']

# Create lag features for the residuals for a 90-day sliding window
def create_lag_features(data, lags, target_col):
    for lag in range(1, lags + 1):
        data[f'lag_{lag}'] = data[target_col].shift(lag)
    return data

# Create lag features for the residuals
df = create_lag_features(df, lags=90, target_col='residuals')

# Drop rows with NaN values caused by shifting (lags)
df.dropna(inplace=True)

# Prepare features (X) and target (y) for XGBoost (we are predicting the residuals)
X = df.drop(columns=['ds', 'y', 'residuals'])
y = df['residuals']

# Split the data into training and testing sets (80% training, 20% testing)
train_size = int(len(df) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Hyperparameter tuning using GridSearchCV for XGBoost
param_grid = {
    'learning_rate': [0.0001, 0.001, 0.01],
    'max_depth': [10, 15, 20],
    'n_estimators': [500, 1000, 1500],
    'min_child_weight': [1, 5, 10],
}

xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train_scaled, y_train)

# Best parameters from GridSearch
print(f'Best parameters from GridSearchCV: {grid_search.best_params_}')

# Use the best parameters to train the XGBoost model
xgb_model = xgb.XGBRegressor(**grid_search.best_params_)
xgb_model.fit(X_train_scaled, y_train)

# Predict the residuals on the test set
residuals_pred = xgb_model.predict(X_test_scaled)

# Combine Prophet's predictions with XGBoost's predictions of the residuals
prophet_forecast_test = prophet_model.predict(df[['ds'] + regressors].iloc[train_size:])
prophet_test_predictions = prophet_forecast_test['yhat']

# Final predictions are Prophet's predictions + XGBoost's predicted residuals
final_predictions = prophet_test_predictions + residuals_pred

# Calculate the RMSE to evaluate the combined model's performance
actual_test_values = df['y'].iloc[train_size:]
rmse = np.sqrt(mean_squared_error(actual_test_values, final_predictions))
print(f'Root Mean Squared Error (RMSE) of the hybrid model: {rmse}')

# Forecasting future prices
# Forecast using Prophet for the next 90 days
future_dates = pd.date_range(start=df['ds'].max(), periods=90, freq='B')
future = pd.DataFrame(future_dates, columns=['ds'])

# Use the last available regressor values for the future predictions
last_regressor_values = df[regressors].iloc[-1].to_dict()
for regressor in regressors:
    future[regressor] = last_regressor_values[regressor]

# Prophet forecast for the future dates
prophet_future_forecast = prophet_model.predict(future)

# Function to make future predictions of residuals using XGBoost
def make_future_residual_predictions(last_sequence, model, num_predictions=90):
    future_residuals = []
    current_seq = last_sequence
    
    for _ in range(num_predictions):
        future_pred_residual = model.predict(np.array([current_seq]))
        future_residuals.append(future_pred_residual[0])
        
        current_seq = np.roll(current_seq, -1)
        current_seq[-1] = future_pred_residual

    return future_residuals

# Predict residuals for the next 90 days
last_sequence = np.array(X.iloc[-1, :])
future_residuals_pred = make_future_residual_predictions(last_sequence, xgb_model, num_predictions=90)

# Combine Prophet's future predictions with XGBoost's residuals
future_predictions = prophet_future_forecast['yhat'] + future_residuals_pred

# Plot actual, hybrid model predictions, and future predictions
plt.figure(figsize=(14, 7))

# Plot actual values from the test set
plt.plot(df['ds'].iloc[train_size:], actual_test_values, label='Actual Gold Price', color='blue')

# Plot hybrid model predictions (Prophet + XGBoost) for the test set
plt.plot(df['ds'].iloc[train_size:], final_predictions, label='Hybrid Model Predictions (Test Set)', color='red')

# Plot future forecasted values (Hybrid Model)
plt.plot(future_dates, future_predictions, label='Future Forecasted Gold Price (Hybrid)', color='green')

plt.title('Gold Price Forecast (Hybrid: Prophet + XGBoost)')
plt.xlabel('Date')
plt.ylabel('Gold Price (LKR)')
plt.legend()
plt.show()

# --- Additional Plotting for Monthly and Weekly Forecasts ---

# Set 'ds' as the index for the future DataFrame to allow resampling
future.set_index('ds', inplace=True)

# Monthly Forecast
monthly_future = future.resample('M').first().reset_index()  # Resample monthly and reset index
monthly_forecast = prophet_model.predict(monthly_future)

plt.figure(figsize=(14, 7))
plt.plot(monthly_forecast['ds'], monthly_forecast['yhat'], label='Monthly Forecast', color='purple')
plt.title('Monthly Gold Price Forecast (Prophet)')
plt.xlabel('Date')
plt.ylabel('Gold Price (LKR)')
plt.legend()
plt.show()

# Weekly Forecast
weekly_future = future.resample('W').first().reset_index()  # Resample weekly and reset index
weekly_forecast = prophet_model.predict(weekly_future)

plt.figure(figsize=(14, 7))
plt.plot(weekly_forecast['ds'], weekly_forecast['yhat'], label='Weekly Forecast', color='orange')
plt.title('Weekly Gold Price Forecast (Prophet)')
plt.xlabel('Date')
plt.ylabel('Gold Price (LKR)')
plt.legend()
plt.show()

# Save the results to a CSV file
combined_results = pd.DataFrame({
    'Date': future_dates,
    'Prophet_Prediction': prophet_future_forecast['yhat'],
    'XGBoost_Residuals': future_residuals_pred,
    'Hybrid_Prediction': future_predictions
})

combined_results.to_csv('hybrid_forecast_results_v02.csv', index=False)
print("Forecasted and predicted values have been saved to 'hybrid_forecast_results_v02.csv'.")


In [ ]:
import joblib
import os

# Create a directory to save the model files
model_dir = "model_files"
os.makedirs(model_dir, exist_ok=True)

# Save the Prophet model using joblib
joblib.dump(prophet_model, os.path.join(model_dir, "prophet_model.pkl"))

# Save the XGBoost model
xgb_model.save_model(os.path.join(model_dir, "xgb_model.json"))

# Save the StandardScaler
joblib.dump(scaler, os.path.join(model_dir, "scaler.pkl"))

# Save the regressor names
joblib.dump(regressors, os.path.join(model_dir, "regressors.pkl"))

print("All necessary files for prediction have been saved in the 'model_files' directory.")

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from prophet import Prophet
import joblib
import logging
import os

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load the dataset
df = pd.read_csv('df_interpolated (3).csv')
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# Rename columns for Prophet
df = df.rename(columns={'date': 'ds', 'gold_lkr': 'y'})

# Log dataset information
logging.info(f"Dataset loaded with {len(df)} rows.")

# Directory where models are saved
model_dir = "model_files"

# Load the saved Prophet model
prophet_model = joblib.load(os.path.join(model_dir, "prophet_model.pkl"))
logging.info("Prophet model loaded successfully.")

# Load the saved XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.load_model(os.path.join(model_dir, "xgb_model.json"))
logging.info("XGBoost model loaded successfully.")

# Load the saved StandardScaler
scaler = joblib.load(os.path.join(model_dir, "scaler.pkl"))
logging.info("Scaler loaded successfully.")

# Load the regressors
regressors = joblib.load(os.path.join(model_dir, "regressors.pkl"))
logging.info("Regressors loaded successfully: %s", regressors)

# Prepare the regressor variables for the entire dataset
df_forecast = prophet_model.predict(df[['ds'] + regressors])
logging.info("Prophet model made predictions for the entire dataset.")

# Create lag features for the residuals
def create_lag_features(data, lags, target_col):
    for lag in range(1, lags + 1):
        data[f'lag_{lag}'] = data[target_col].shift(lag)
    return data

# Compute residuals (y - Prophet predictions)
df['residuals'] = df['y'] - df_forecast['yhat']

# Create 90-day lag features
df = create_lag_features(df, lags=90, target_col='residuals')

# Drop rows with NaN values caused by shifting (lags)
df.dropna(inplace=True)
logging.info(f"After creating lag features, dataset has {len(df)} rows.")

# Prepare features (X) for XGBoost
X = df.drop(columns=['ds', 'y', 'residuals'])

# Scale the features
X_scaled = scaler.transform(X)
logging.info("Features have been scaled for XGBoost model.")

# Predict residuals using the XGBoost model
residuals_pred = xgb_model.predict(X_scaled)
logging.info("XGBoost model made residual predictions.")

# Adjust the df_forecast to account for the 90-day lag
df_forecast_lagged = df_forecast.iloc[90:].reset_index(drop=True)

# Ensure the number of Prophet predictions matches the number of residual predictions
if len(df_forecast_lagged['yhat']) != len(residuals_pred):
    logging.error("Length mismatch: Prophet predictions vs. XGBoost residuals")
    raise ValueError(f"Length mismatch: Prophet predictions ({len(df_forecast_lagged['yhat'])}) vs. XGBoost residuals ({len(residuals_pred)})")

# Combine Prophet's predictions with XGBoost's predicted residuals
final_predictions = df_forecast_lagged['yhat'] + residuals_pred
logging.info("Final hybrid predictions (Prophet + XGBoost) computed.")

# Save the forecasted values along with the respective dates (adjust dates for the lag)
forecast_results = pd.DataFrame({
    'Date': df['ds'].iloc[90:].reset_index(drop=True),
    'Prophet_Prediction': df_forecast_lagged['yhat'],
    'XGBoost_Residuals': residuals_pred,
    'Hybrid_Prediction': final_predictions
})

# Save the forecast results to CSV
forecast_results.to_csv('forecast_results_FULL_v02.csv', index=False)
logging.info("Forecasted values have been saved to 'hybrid_forecast_results_v02.csv'.")